In [ ]:
# default_exp models.bases.ctr

# CTR Models
> Implementation of CTR base models in Pytorch lightning.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.nb_imports import *
from fastcore.test import *

In [ ]:
#export
import os
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss, accuracy_score

import torch
from torch import nn
from torch.nn import functional as F

from pytorch_lightning import LightningModule

In [ ]:
#exporti
def evaluate_metrics(y_true, y_pred, metrics):
    result = dict()
    for metric in metrics:
        if metric in ['logloss', 'binary_crossentropy']:
            result[metric] = log_loss(y_true, y_pred, eps=1e-7)
        elif metric == 'AUC':
            result[metric] = roc_auc_score(y_true, y_pred)
        elif metric == "ACC":
            y_pred = np.argmax(y_pred, axis=1)
            result[metric] = accuracy_score(y_true, y_pred)
    print('[Metrics] ' + ' - '.join('{}: {:.6f}'.format(k, v) for k, v in result.items()))
    return result

In [ ]:
#export
class CTRModel(LightningModule):
    def __init__(self, 
                 feature_map, 
                 model_id="CTRModel",
                 optimizer='adamw',
                 learning_rate = 0.003,
                 **kwargs):
        super().__init__()
        self._feature_map = feature_map
        self.model_id = model_id
        self.optimizer = optimizer
        self.learning_rate = learning_rate
        self.model_dir = os.path.join(kwargs["model_root"], feature_map.dataset_id)
        self.checkpoint = os.path.abspath(os.path.join(self.model_dir, self.model_id + "_model.ckpt"))
        self._validation_metrics = kwargs["metrics"]
        self._verbose = kwargs["verbose"]

    def forward(self, users, items):
        raise NotImplementedError

    def training_step(self, batch, batch_idx):
        features, y_true = batch
        y_pred = self(features)

        y_pred = y_pred.view(-1,1).squeeze()
        y_true = y_true.float()

        loss = self.loss_fn(y_pred, y_true)

        return {
            "loss": loss,
            "y_pred": y_pred.detach(),
        }

    def training_epoch_end(self, outputs):
        # This function recevies as parameters the output from "training_step()"
        # Outputs is a list which contains a dictionary like:
        # [{'pred':x,'target':x,'loss':x}, {'pred':x,'target':x,'loss':x}, ...]
        pass

    def validation_step(self, batch, batch_idx):
        features, y_true = batch
        y_pred = self(features)

        y_pred = np.array(y_pred.cpu().numpy().reshape(-1), np.float64)
        y_true = np.array(y_true.cpu().numpy().reshape(-1), np.float64)
        val_logs = evaluate_metrics(y_true, y_pred, self._validation_metrics)
        self.log("Val Metrics", val_logs, prog_bar=True)

        return {
            "y_pred": y_pred,
        }

    def validation_epoch_end(self, outputs):
        pass

    def test_step(self, batch, batch_idx):
        features, y_true = batch
        y_pred = self(features)

        y_pred = np.array(y_pred.cpu().numpy().reshape(-1), np.float64)
        y_true = np.array(y_true.cpu().numpy().reshape(-1), np.float64)
        test_logs = evaluate_metrics(y_true, y_pred, self._validation_metrics)
        self.log("Test Metrics", test_logs, prog_bar=True)

        return {
            "y_pred": y_pred,
        }

    def test_epoch_end(self, outputs):
        pass

    def configure_optimizers(self):
        if self.optimizer == 'adamw':
            return torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        elif self.optimizer == 'adam':
            return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        elif self.optimizer == 'sgd':
            return torch.optim.SGD(self.parameters(), lr=self.learning_rate)
        else:
            raise ValueError(f'Invalid optimizer type: {self.optimizer}')

    def loss_fn(self, y_pred, y_true):
        return F.binary_cross_entropy(y_pred, y_true, reduction='mean')        

    def init_weights(self, embedding_initializer=None):
        def _initialize(m):
            if type(m) == nn.ModuleDict:
                for k, v in m.items():
                    if type(v) == nn.Embedding:
                        if "pretrained_emb" in self._feature_map.feature_specs[k]: # skip pretrained
                            continue
                        if embedding_initializer is not None:
                            try:
                                initializer = embedding_initializer.replace("(", "(v.weight,")
                                eval(initializer)
                            except:
                                raise NotImplementedError("embedding_initializer={} is not supported."\
                                                          .format(embedding_initializer))
                        else:
                            nn.init.xavier_normal_(v.weight)
            if type(m) == nn.Linear:
                nn.init.xavier_normal_(m.weight)
                if m.bias is not None:
                    m.bias.data.fill_(0)
        self.apply(_initialize)

    def get_final_activation(self, task="binary_classification"):
        if task == "binary_classification":
            return nn.Sigmoid()
        elif task == "multi_classification":
            return nn.Softmax(dim=-1)
        elif task == "regression":
            return None
        else:
            raise NotImplementedError("task={} is not supported.".format(task))

> **References**
> - FuxiCTR Project, https://github.com/RecoHut-Stanzas/S516304.

In [ ]:
#hide
%reload_ext watermark
%watermark -a "Sparsh A." -m -iv -u -t -d -p pytorch_lightning

Author: Sparsh A.

Last updated: 2022-01-11 14:29:29

pytorch_lightning: 1.5.8

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 5.4.144+
Machine     : x86_64
Processor   : x86_64
CPU cores   : 2
Architecture: 64bit

IPython   : 5.5.0
PIL       : 7.1.2
numpy     : 1.19.5
torch     : 1.10.0+cu111
matplotlib: 3.2.2



# Prototype